In [1]:
import pandas as pd 
# !pip install requests
import requests 
import numpy as np
import lxml
from pprint import pprint
url = "https://transfermarket.p.rapidapi.com/statistic/list-most-valuable-clubs"

In [2]:
headers = {
	"X-RapidAPI-Key": "900cc08f36msh48bec253bfe456bp100148jsn8c5ab8de0a04",
	"X-RapidAPI-Host": "transfermarket.p.rapidapi.com"
}
euro_rankings = pd.read_html('https://fbref.com/en/')[0]
euro_rankings

for index, row in euro_rankings.iterrows():
   team_name = row['Squad']
   querystring = {"query":team_name,"domain":"de"}
   response = requests.request("GET", url, headers=headers, params=querystring)

    
api_response = response.json()

#make a dataframe from the api_response
response_df = pd.DataFrame(api_response['teams'])

#drop unneeded columns 
response_df.drop(['number', 'countryImage','clubImage', 'marketValue', 'marketValueNumeral'], axis=1, inplace=True)

#rename columns
response_df.rename(columns={'clubName':'Team', 'marketValueUnformatted':'Market Value', 'countryName':'Country'}, inplace=True)

response_df.rename(columns={'competitionID': 'League ID', 
                            'competitionName': 'League', 
                            'avgAge': 'Average Age', 
                            'marketValueCurrency': 'Currency'}, inplace=True)

KeyError: 'teams'

In [ ]:
# Collect Lat/Long then merge to response_df
# Enter 'pip install geopy' to command to install geopy

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-app")
countries = response_df['Country'].unique()
lat=[]
long=[]

for c in countries:
    location = geolocator.geocode(c)
    if location:
        lat.append(location.latitude)
        long.append(location.longitude)
    else:
        lat.append(None)
        long.append(None)
countries_df = pd.DataFrame({"Country":countries,
                            "Lat":lat,
                            "Long":long})


In [ ]:
countries_df

In [ ]:
#Merge main data with lat/long
response_df=pd.merge(response_df,countries_df,how='left',on='Country')
response_df.head()

In [ ]:
response_df['League'].unique()

In [ ]:
# Collect performance data from fbef website
Ligue_1_League_ranking  = pd.read_html('https://fbref.com/en/comps/13/Ligue-1-Stats')[0]
Serie_A_rankings = pd.read_html('https://fbref.com/en/comps/11/Serie-A-Stats')[0]
Bundesliga_rankings = pd.read_html('https://fbref.com/en/comps/20/Bundesliga-Stats')[0]
Laliga_rankings = pd.read_html('https://fbref.com/en/comps/12/La-Liga-Stats')[0]
Premier_rankings = pd.read_html('https://fbref.com/en/comps/9/Premier-League-Stats')[0]
Liga_Portugal_rankings = pd.read_html('https://fbref.com/en/comps/32/Primeira-Liga-Stats')[0]
Eredivisie_rankings = pd.read_html('https://fbref.com/en/comps/23/Eredivisie-Stats')[0]
Super_Lig_rankings = pd.read_html('https://fbref.com/en/comps/26/Super-Lig-Stats')[0]
Jupiler_Pro_League_rankings = pd.read_html('https://fbref.com/en/comps/37/Belgian-First-Division-A-Stats')[0]
Campeonato_rankings = pd.read_html('https://fbref.com/en/comps/24/Serie-A-Stats')[0]
Scottish_Premiership_rankings = pd.read_html('https://fbref.com/en/comps/40/Scottish-Premiership-Stats')[0]
Championship_rankings = pd.read_html('https://fbref.com/en/comps/10/Championship-Stats')[0]
Super_League_1_rankings = pd.read_html('https://fbref.com/en/comps/27/Super-League-Greece-Stats')[0]
Campeonato_Mineiro_rankings=pd.read_html('https://fbref.com/en/comps/38/Serie-B-Stats')[0]
Liga_Professional_de_Futbol_rankings=pd.read_html('https://fbref.com/en/comps/21/Primera-Division-Stats')[0]
Supersport_HNL_rankings=pd.read_html('https://fbref.com/en/comps/63/1-HNL-Stats')[0]
Austria_rankings = pd.read_html('https://fbref.com/en/comps/56/Austrian-Bundesliga-Stats')[0]
Russia_rankings = pd.read_html('https://fbref.com/en/comps/30/Russian-Premier-League-Stats')[0]

In [ ]:
# Union to 1 DataFrame
performance_df = pd.concat([Ligue_1_League_ranking,
                           Serie_A_rankings,
                           Bundesliga_rankings,
                           Laliga_rankings,
                           Premier_rankings,
                           Liga_Portugal_rankings,
                           Eredivisie_rankings,
                           Super_Lig_rankings,
                           Jupiler_Pro_League_rankings,
                           Campeonato_rankings,
                           Scottish_Premiership_rankings,
                           Championship_rankings,
                           Super_League_1_rankings,
                           Campeonato_Mineiro_rankings,
                           Liga_Professional_de_Futbol_rankings,
                           Supersport_HNL_rankings,
                           Austria_rankings,
                           Russia_rankings])

# Select columns
performance_df=performance_df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP']]

performance_df

In [ ]:
response_df2 = response_df.loc[(response_df['League ID'] != 'L1')&
    (response_df['League ID'] != 'GB1')&
    (response_df['League ID'] != 'ES1')&
    (response_df['League ID'] != 'FR1')&
    (response_df['League ID'] != 'IT1'),:]
for r in response_df2.Team.unique():
    print(r)

In [ ]:
for r in performance_df.Squad.unique():
    print(r)

In [ ]:
# Clean performance_df['Squad'] to match with response_df['Team'] before merging
response_df['Team']=[row.replace('1.FC Köln','Köln') for row in response_df['Team']]
response_df['Team']=[row.replace('1.FSV Mainz 05','Mainz 05') for row in response_df['Team']]
response_df['Team']=[row.replace('AC Florenz','Fiorentina') for row in response_df['Team']]
response_df['Team']=[row.replace('AC Mailand','Milan') for row in response_df['Team']]
response_df['Team']=[row.replace('AS Monaco','Monaco') for row in response_df['Team']]
response_df['Team']=[row.replace('AS Rom','Roma') for row in response_df['Team']]
response_df['Team']=[row.replace('Aston Villa','Aston Villa') for row in response_df['Team']]
response_df['Team']=[row.replace('Atal. Bergamo','Atalanta') for row in response_df['Team']]
response_df['Team']=[row.replace('Athletic Bilbao','Athletic Club') for row in response_df['Team']]
response_df['Team']=[row.replace('Atlético Madrid','Atlético Madrid') for row in response_df['Team']]
response_df['Team']=[row.replace('B. Leverkusen','Leverkusen') for row in response_df['Team']]
response_df['Team']=[row.replace('Bayern München','Bayern Munich') for row in response_df['Team']]
response_df['Team']=[row.replace('Betis Sevilla','Sevilla') for row in response_df['Team']]
response_df['Team']=[row.replace('Bor. Dortmund','Dortmund') for row in response_df['Team']]
response_df['Team']=[row.replace("Bor. M'gladbach","M'Gladbach") for row in response_df['Team']]
response_df['Team']=[row.replace('Bournemouth','Bournemouth') for row in response_df['Team']]
response_df['Team']=[row.replace('Brighton','Brighton') for row in response_df['Team']]
response_df['Team']=[row.replace('CA Osasuna','Osasuna') for row in response_df['Team']]
response_df['Team']=[row.replace('Celta Vigo','Celta Vigo') for row in response_df['Team']]
response_df['Team']=[row.replace('Crystal Palace','Crystal Palace') for row in response_df['Team']]
response_df['Team']=[row.replace('E. Frankfurt','Eint Frankfurt') for row in response_df['Team']]
response_df['Team']=[row.replace('Esp. Barcelona','Barcelona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Arsenal','Arsenal') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Augsburg','Augsburg') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Barcelona','Barcelona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Bologna','Bologna') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Brentford','Brentford') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Chelsea','Chelsea') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Everton','Everton') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Fulham','Fulham') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Getafe','Getafe') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Girona','Girona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Nantes','Nantes') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Sevilla','Sevilla') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Turin','Torino') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Valencia','Valencia') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Villarreal','Villarreal') for row in response_df['Team']]
response_df['Team']=[row.replace('Hertha BSC','Hertha BSC') for row in response_df['Team']]
response_df['Team']=[row.replace('Inter Mailand','Inter') for row in response_df['Team']]
response_df['Team']=[row.replace('Juventus Turin','Juventus') for row in response_df['Team']]
response_df['Team']=[row.replace('Lazio Rom','Lazio') for row in response_df['Team']]
response_df['Team']=[row.replace('Leeds United','Leeds United') for row in response_df['Team']]
response_df['Team']=[row.replace('Leicester City','Leicester City') for row in response_df['Team']]
response_df['Team']=[row.replace('Liverpool','Liverpool') for row in response_df['Team']]
response_df['Team']=[row.replace('LOSC Lille','Lille') for row in response_df['Team']]
response_df['Team']=[row.replace('Manchester City','Manchester City') for row in response_df['Team']]
response_df['Team']=[row.replace('Manchester Utd.','Manchester Utd') for row in response_df['Team']]
response_df['Team']=[row.replace('Montpellier HSC','Montpellier') for row in response_df['Team']]
response_df['Team']=[row.replace('Monza','Monza') for row in response_df['Team']]
response_df['Team']=[row.replace('Newcastle Utd.','Newcastle Utd') for row in response_df['Team']]
response_df['Team']=[row.replace('Nottm Forest',"Nott'ham Forest") for row in response_df['Team']]
response_df['Team']=[row.replace('OGC Nizza','Nice') for row in response_df['Team']]
response_df['Team']=[row.replace('Ol. Marseille','Marseille') for row in response_df['Team']]
response_df['Team']=[row.replace('Olympique Lyon','Lyon') for row in response_df['Team']]
response_df['Team']=[row.replace('Paris SG','Paris S-G') for row in response_df['Team']]
response_df['Team']=[row.replace('RB Leipzig','RB Leipzig') for row in response_df['Team']]
response_df['Team']=[row.replace('RC Lens','Lens') for row in response_df['Team']]
response_df['Team']=[row.replace('Real Madrid','Real Madrid') for row in response_df['Team']]
response_df['Team']=[row.replace('Real Sociedad','Real Sociedad') for row in response_df['Team']]
response_df['Team']=[row.replace('Rennes','Rennes') for row in response_df['Team']]
response_df['Team']=[row.replace('SC Freiburg','Freiburg') for row in response_df['Team']]
response_df['Team']=[row.replace('Southampton','Southampton') for row in response_df['Team']]
response_df['Team']=[row.replace('SSC Neapel','Napoli') for row in response_df['Team']]
response_df['Team']=[row.replace('Tottenham','Tottenham') for row in response_df['Team']]
response_df['Team']=[row.replace('TSG Hoffenheim','Hoffenheim') for row in response_df['Team']]
response_df['Team']=[row.replace('UD Almería','Almería') for row in response_df['Team']]
response_df['Team']=[row.replace('Udinese Calcio','Udinese') for row in response_df['Team']]
response_df['Team']=[row.replace('Union Berlin','Union Berlin') for row in response_df['Team']]
response_df['Team']=[row.replace('US Salernitana','Salernitana') for row in response_df['Team']]
response_df['Team']=[row.replace('US Sassuolo','Sassuolo') for row in response_df['Team']]
response_df['Team']=[row.replace('VfB Stuttgart','Stuttgart') for row in response_df['Team']]
response_df['Team']=[row.replace('VfL Wolfsburg','Wolfsburg') for row in response_df['Team']]
response_df['Team']=[row.replace('West Ham Utd.','West Ham') for row in response_df['Team']]
response_df['Team']=[row.replace('Wolverhampton','Wolves') for row in response_df['Team']]
response_df['Team']=[row.replace('Benfica','Benfica') for row in response_df['Team']]
response_df['Team']=[row.replace('Ajax Amsterdam','Ajax') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Porto','Porto') for row in response_df['Team']]
response_df['Team']=[row.replace('Sporting','Sporting CP') for row in response_df['Team']]
response_df['Team']=[row.replace('RB Salzburg','RB Salzburg') for row in response_df['Team']]
response_df['Team']=[row.replace('PSV Eindhoven','PSV Eindhoven') for row in response_df['Team']]
response_df['Team']=[row.replace('Fenerbahce','Fenerbahçe') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Brügge','Club Brugge') for row in response_df['Team']]
response_df['Team']=[row.replace('Galatasaray','Galatasaray') for row in response_df['Team']]
response_df['Team']=[row.replace('Flamengo','Flamengo') for row in response_df['Team']]
response_df['Team']=[row.replace('Zenit S-Pb','Zenit') for row in response_df['Team']]
response_df['Team']=[row.replace('Feyenoord','Feyenoord') for row in response_df['Team']]
response_df['Team']=[row.replace('Trabzonspor','Trabzonspor') for row in response_df['Team']]
response_df['Team']=[row.replace('Glasgow Rangers','Rangers') for row in response_df['Team']]
response_df['Team']=[row.replace('Palmeiras','Palmeiras') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Watford','Watford') for row in response_df['Team']]
response_df['Team']=[row.replace('Besiktas','Beşiktaş') for row in response_df['Team']]
response_df['Team']=[row.replace('Atlético-MG','Atlético Mineiro') for row in response_df['Team']]
response_df['Team']=[row.replace('SC Braga','Braga') for row in response_df['Team']]
response_df['Team']=[row.replace('Celtic Glasgow','Celtic') for row in response_df['Team']]
response_df['Team']=[row.replace('Olymp. Piräus','Olympiacos') for row in response_df['Team']]
response_df['Team']=[row.replace('Norwich City','Norwich City') for row in response_df['Team']]
response_df['Team']=[row.replace('KRC Genk ','Genk') for row in response_df['Team']]
response_df['Team']=[row.replace('River Plate','River Plate') for row in response_df['Team']]
response_df['Team']=[row.replace('Dinamo Zagreb','Dinamo Zagreb') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Burnley','Burnley') for row in response_df['Team']]

response_df.head()

In [ ]:
# Merge performance_df with response_df through Squad and Team
response_df=pd.merge(response_df,performance_df,how='left',left_on='Team',right_on='Squad')
response_df

In [ ]:
# Drop Squad column as contain same value as Team
response_df.drop(['Squad'], axis=1, inplace=True)

In [ ]:
response_df.head()

In [ ]:
response_df.info()

In [ ]:
# Save reponse_df to a csv
response_df.to_csv('cleaned_data.csv',index=False)